In [1]:
import sys
print(sys.executable)
print(sys.version)
print(sys.version_info)

/usr/local/opt/python@3.9/bin/python3.9
3.9.2 (default, Feb 24 2021, 13:26:09) 
[Clang 12.0.0 (clang-1200.0.32.29)]
sys.version_info(major=3, minor=9, micro=2, releaselevel='final', serial=0)


### Managed Kafka Setup

[rhoas binary](https://github.com/redhat-developer/app-services-cli/releases)

`brew install jq`

`rhoas login`

`rhoas kafka list`

`rhoas kafka topic list`

`rhoas kafka create shipwars`

`rhoas kafka describe shipwars | jq '.status'`

`rhoas kafka topic create shipwars-matches --partitions 3`

`rhoas kafka topic create shipwars-players --partitions 3`

`rhoas kafka topic create shipwars-attacks --partitions 3`

`rhoas kafka topic create shipwars-bonuses --partitions 3`

`rhoas kafka topic create shipwars-results --partitions 3`

`rhoas kafka topic create test --partitions 3`

`rhoas kafka topic list`

We only need the "test" topic for this example code, everything else was to be compatible with the shipwars demo

`rhoas service-account create --name "myserviceaccount" --file-format json`

`export KAFKA_BOOTSTRAP_SERVER=$(rhoas kafka describe shipwars | jq -r '.bootstrap_server_host')`

`export KAFKA_USERNAME=$(jq -r '.clientID' credentials.json)`

`export KAFKA_PASSWORD=$(jq -r '.clientSecret' credentials.json)`

`export KAFKA_TOPIC=test`

`export KAFKA_CONSUMER_GROUP=notebook-consumer`


In [2]:
!pip install kafka-python


DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Homebrew or Linuxbrew Python, please see discussion at https://github.com/Homebrew/homebrew-core/issues/76621


In [3]:
!pip list

Package             Version
------------------- -------
appnope             0.1.2
argon2-cffi         20.1.0
attrs               21.2.0
backcall            0.2.0
bleach              4.0.0
cffi                1.14.6
debugpy             1.4.1
decorator           5.0.9
defusedxml          0.7.1
entrypoints         0.3
ipykernel           6.2.0
ipython             7.26.0
ipython-genutils    0.2.0
ipywidgets          7.6.3
jedi                0.18.0
Jinja2              3.0.1
jsonschema          3.2.0
jupyter             1.0.0
jupyter-client      6.1.12
jupyter-console     6.4.0
jupyter-core        4.7.1
jupyterlab-pygments 0.1.2
jupyterlab-widgets  1.0.0
kafka-python        2.0.2
MarkupSafe          2.0.1
matplotlib-inline   0.1.2
mistune             0.8.4
nbclient            0.5.4
nbconvert           6.1.0
nbformat            5.1.3
nest-asyncio        1.5.1
notebook            6.4.3
packaging           21.0
pandocfilters       1.4.3
parso               0.8.2
pexpect             4.8.0
pickl

In [4]:
import os

The following env variables should be set prior to running 'jupyter notebook' if using localhost

In [5]:
KAFKA_BOOTSTRAP_SERVER = os.environ.get('KAFKA_BOOTSTRAP_SERVER')
KAFKA_USERNAME = os.environ.get('KAFKA_USERNAME')
KAFKA_PASSWORD = os.environ.get('KAFKA_PASSWORD')
KAFKA_TOPIC = 'test'
KAFKA_CONSUMER_GROUP = 'notebook-consumer'

In [6]:
print(KAFKA_BOOTSTRAP_SERVER)
print(KAFKA_USERNAME)
print(KAFKA_PASSWORD)
print(KAFKA_TOPIC)
print(KAFKA_CONSUMER_GROUP)

shipwars--x-bklrrwx-pasloxste-ggcxtf.bf2.kafka.rhcloud.com:443
srvc-acct-2b021695-181b-4a41-b7d3-c02b457ea179
e8936e08-127f-4b09-a3b1-971576101de4
test
notebook-consumer


In [7]:
from kafka import KafkaConsumer


def create_consumer_print():
    consumer = KafkaConsumer(KAFKA_TOPIC,
                             group_id=KAFKA_CONSUMER_GROUP,
                             bootstrap_servers=[KAFKA_BOOTSTRAP_SERVER],
                             security_protocol='SASL_SSL',
                             sasl_mechanism='PLAIN',
                             sasl_plain_username=KAFKA_USERNAME,
                             sasl_plain_password=KAFKA_PASSWORD,
                             auto_offset_reset='earliest',
                             api_version_auto_timeout_ms=30000,
                             request_timeout_ms=450000)
    print(f'Subscribed to "{KAFKA_BOOTSTRAP_SERVER}" consuming topic "{KAFKA_TOPIC}"...')

    try:
        for record in consumer:
            msg = record.value.decode('utf-8')
            topic = record.topic
            print(('Received the following message on the '
                   f'topic "{topic}": {msg}'))

    finally:
        print("Closing consumer...")
        consumer.close()
    print("Kafka consumer stopped.")

In [ ]:
try:
    create_consumer_print()
except KeyboardInterrupt:
    print('Stopped')

Subscribed to "shipwars--x-bklrrwx-pasloxste-ggcxtf.bf2.kafka.rhcloud.com:443" consuming topic "test"...
Received the following message on the topic "test": one
Received the following message on the topic "test": two
Received the following message on the topic "test": three
Received the following message on the topic "test": four
Received the following message on the topic "test": five
Received the following message on the topic "test": six
